In [1]:
import sys
import struct
import ctypes

https://toster.ru/q/565035

https://habr.com/ru/post/193890/  

https://intellipaat.com/blog/tutorial/c-tutorial/c-data-types/

https://hackmd.io/@xff9N3eQTLSL4Trq-6setg/ByMHBMjFe?type=view#08-Debugging-Python-Objects

https://stackoverflow.com/questions/49153714/why-is-the-size-of-2%E2%81%B6%C2%B3-36-bytes-but-2%E2%81%B6%C2%B3-1-is-only-24-bytes/49154022

In [2]:
sys.version
# есть разница, 64бит или 32бит. На 32бит размеры меньше: флоат - 12, инт - 14

'3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]'

int - начиная с 28 байт, + 4 байт на каждый новый блок (2**30)

float и ноль - 24 байта

str - пустая - 49 байт, + добавляются байты за каждый знак

In [444]:
i = 2**30-1
print(sys.getsizeof(i))
i = 2**30
print(sys.getsizeof(i))
i = 2**30+1
print(sys.getsizeof(i))
i = 2**60
print(sys.getsizeof(i))
i = -22
print(sys.getsizeof(i))

28
32
32
36
28


In [399]:
#float
obj_f = 1.1
print(sys.getsizeof(obj_f))
obj_f = 0
print(sys.getsizeof(obj_f))
obj_f = 222.9475620022
print(sys.getsizeof(obj_f))
obj_f = 0.9475620022
print(sys.getsizeof(obj_f))

24
24
24
24


In [398]:
#string
obj_str = ''
print(sys.getsizeof(obj_str))
obj_str = ' '
print(sys.getsizeof(obj_str))
obj_str = 't'
print(sys.getsizeof(obj_str))
obj_str = 'т'
print(sys.getsizeof(obj_str))

49
50
54
76


sys.getsizeof(объект[, значение_по_умолчанию])



Возвращает размер объекта в байтах.
Если указано значение по умолчанию, то оно вернется, если объект не предоставляет способа получить размер. В противном случае возникнет исключение TypeError.

Getsizeof() вызывает метод объекта __sizeof__ и добавляет размер дополнительной информации, которая хранится для сборщика мусора, если он используется.

Для простых типов данных (которые не содержат ссылок на другие объекты, и соответственно, не отслеживаются сборщиком мусора), значение sys.getsizeof равно значению, возвращаемого методом __sizeof__().

Встроенная функция id() возвращает адрес памяти, где хранится начала объекта (сам объект является C структурой)

Чтобы считать данные по адресу памяти нужно воспользоваться функцией string_at из модуля ctypes. 

ctypes.string_at(address, size=-1)
This function returns the C string starting at memory address "address" as a bytes object. If size is specified, it is used as size, otherwise the string is assumed to be zero-terminated.

In [403]:
# Теперь попробуем считать данные по адресу, который вернул нам id():
obj = 2
print(ctypes.string_at(id(obj), 28) )
obj = 1
print(ctypes.string_at(id(obj), 28) )
obj = 1152921504606846976 # 2**60
print(ctypes.string_at(id(obj), 36) )

b'O\x08\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00'
b'\xb4\x14\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00'
b'\x02\x00\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00'


Для того чтобы представить вывод в значения, удобные для восприятия, воспользуемся еще одним модулем. Здесь нам поможет функция unpack() из модуля struct.

struct
Этот модуль производит преобразование между значениями Python и структурами на C, представленными в виде строк.
This module performs conversions between Python values and C structs represented as Python bytes objects.

struct.unpack(формат, строка)
Разбирает строку в соответствие с данным форматов. Всегда возвращает кортеж, даже если строка содержит только один элемент. 
####  Строка должна содержать в точности то количество информации, как описано форматом.

Since Python 3 there is no longer simple integer type, and all integers are represented as a bignum.

In [450]:
obj = 501
struct.unpack('lPnl', ctypes.string_at(id(obj), 28)) 

(2, 1383436864, 1, 501)

In [407]:
print(sys.getrefcount(obj))
print(id(type(obj)))

2
1383436864


по адресу в памяти выводится следующая информация:
- количество ссылок на объект(эта информация занимает 4 байта)
- id типа объекта (8 байт)
- к какому "блоку" принадлежит число (base 2**30 numeral system)  (1 - 2 ** 30, 2 ** 30 - 2 ** 60, и т.д) (8 байт)
- само число (от 4 байт, в зависимости от блока) #inline array of value digits

https://stackoverflow.com/questions/49153714/why-is-the-size-of-2%E2%81%B6%C2%B3-36-bytes-but-2%E2%81%B6%C2%B3-1-is-only-24-bytes/49154022
    

In [459]:
obj = 1152921504606846976 # 2**60
print(struct.unpack('lPnlll', ctypes.string_at(id(obj), 36) )) # не знаю, как вывести последнее число

(2, 1383436864, 3, 0, 0, 1)


In [448]:
struct.unpack('lPnl', ctypes.string_at(id(-4), 28)) 

(36, 1383436864, -1, 4)

In [271]:
ctypes.string_at(id(-1180591620717411303426), 36)

b'\x01\x00\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\xfd\xff\xff\xff\xff\xff\xff\xff\x02\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00'

In [275]:
ctypes.string_at(id(-1180591620717411303428), 36)

b'\x01\x00\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\xfd\xff\xff\xff\xff\xff\xff\xff\x04\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00'

In [543]:
obj = 2**22
print(obj)
print(ctypes.string_at(id(obj), 28))
print(sys.getsizeof(obj))

print(struct.unpack('lPnl', ctypes.string_at(id(obj), 28)) )
print(struct.unpack('nl', ctypes.string_at(b'\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00', 12)) )
print(struct.unpack('l', ctypes.string_at(b'\x00\x00@\x00', 4)) )

4194304
b'\x01\x00\x00\x00\x00\x00\x00\x00@\x92uR\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00'
28
(1, 1383436864, 1, 4194304)
(1, 4194304)
(4194304,)


In [47]:
#  Floating point numbers are usually implemented using double in C;
# information about the precision and internal representation of floating point numbers
# for the machine on which your program is running is available in sys.float_info.
sys.float_info

sys.float_info(max=1.7976931348623157e+308, max_exp=1024, max_10_exp=308, min=2.2250738585072014e-308, min_exp=-1021, min_10_exp=-307, dig=15, mant_dig=53, epsilon=2.220446049250313e-16, radix=2, rounds=1)

If you call id(x) multiple for any x in the range [-5, 256], you will get the same result every time (for the same integer). 
But if you try it for integers outside this range, each one will be different
(a new object is created on the fly every time).

In [51]:
id(101)

1383906656

In [52]:
id(101)

1383906656

In [53]:
id(301)

3252003662928

In [54]:
id(301)

3252003663472

In [ ]:
DESCRIPTION
    Functions to convert between Python values and C structs.
    Python bytes objects are used to hold the data representing the C struct
    and also as format strings (explained below) to describe the layout of data
    in the C struct.
    
    The optional first format char indicates byte order, size and alignment:
      @: native order, size & alignment (default)
      =: native order, std. size & alignment
      <: little-endian, std. size & alignment
      >: big-endian, std. size & alignment
      !: same as >
    
    The remaining chars indicate types of args and must match exactly;
    these can be preceded by a decimal repeat count:
      x: pad byte (no data); c:char; b:signed byte; B:unsigned byte;
      ?: _Bool (requires C99; if not available, char is used instead)
      h:short; H:unsigned short; i:int; I:unsigned int;
      l:long; L:unsigned long; f:float; d:double; e:half-float.
    Special cases (preceding decimal count indicates length):
      s:string (array of char); p: pascal string (with count byte).
    Special cases (only available in native format):
      n:ssize_t; N:size_t;
      P:an integer type that is wide enough to hold a pointer.
    Special case (not in native mode unless 'long long' in platform C):
      q:long long; Q:unsigned long long
    Whitespace between formats is ignored.

In [171]:
struct.unpack('n', ctypes.string_at(b'\xff\xff\xff\xff\xff\xff\xff\xff', 8)) #

(-1,)

In [131]:
struct.unpack('n', ctypes.string_at(b'', 8)) 

(0,)

In [85]:
l = 4,
L = 4,
N = 8,
n = 8,
q = 8,
Q = 8,
P = 8,
d = 8,
e = 2,
h = 2,
c = 1,
x = 1,
? = 1
ln = lln = 16 (l начинает занимать тоже 8 знаков)

In [ ]:
typedef struct {
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;

    long ob_digit[1];
} PyLongObject;

In [383]:
a

2147483648

In [387]:
isinstance(a,int)

True

In [388]:
x = 1
y = x
x = 2
y

1

In [544]:
x = 1
y = x
y = 2
x

1

In [390]:
a = 257
print(id(a))
b = 257
print(id(b))

3252006045872
3252006045232


In [391]:
a = 256
print(id(a))
b = 256
print(id(b))

1383911616
1383911616


In [ ]:
мантисса, экспонента
https://habr.com/ru/post/112953/
http://ru.exploringcomputerarithmetic.org/articles/fp/IEEE-754/basics/computer-representation/  

### bin

In [15]:
print(bin(0))
print(bin(1))
print(bin(4))
print(bin(14))
print(bin(10000))

0b0
0b1
0b100
0b1110
0b10011100010000


In [16]:
def bits(n):
    n += 2**32
    return bin(n)[-32:] # отрезает первую единицу тоже

In [17]:
print(bits(0))
print(bits(1))
print(bits(4))
print(bits(14))
print(bits(100))

00000000000000000000000000000000
00000000000000000000000000000001
00000000000000000000000000000100
00000000000000000000000000001110
00000000000000000000000001100100


In [7]:
print(bits(0))
print(bits(1))
print(bits(4))
print(bits(14))
print(bits(100))

0b100000000000000000000000000000000
0b100000000000000000000000000000001
0b100000000000000000000000000000100
0b100000000000000000000000000001110
0b100000000000000000000000001100100


In [18]:
2**32

4294967296

In [19]:
bin(4294967296)

'0b100000000000000000000000000000000'

In [20]:
len('100000000000000000000000000000000')

33

In [12]:
bin(-4294967297)[-33:]

'100000000000000000000000000000001'

In [3]:
sys.int_info

sys.int_info(bits_per_digit=30, sizeof_digit=4)